In [2]:
from baseline import *
import pickle
with open('dict_data_final.pkl', 'rb') as f:
    dict_data_final = pickle.load(f)

In [109]:
## Parámetros ## 

P = [
    [0, 0.1, 0.5, 0.9, 1.01],
    [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.01],
    [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.01]
]

C = [1, 1.5, 2]

lista = list(dict_data_final.keys())
lista.append('Total')

In [123]:
from torchmetrics import Accuracy 
accuracy = Accuracy(task="multiclass", num_classes=3)
cross_entropy = torch.nn.CrossEntropyLoss()

In [142]:
i, j = -1, -1
for part in P:
    i += 1
    for div in C:
        j += 1
        #### dataframe ####
        df = pd.DataFrame(
            np.zeros((12, 5)),
            index = lista,
            columns=['Accuracy', 'CrossEntropy', 'Fired', 'ModelFired', 'ExpectedFired'],
            dtype=float
        )
        ####################
        
        for key in dict_data_final.keys():
            print(part, div, key)
            x, y = dict_data_final[key]

            grid = Grid(x=x, y=y)
            grid.initialize(part=part, div=div)
            grid.submatrix()
            grid.enlargement_process()
            grid.montecarlo(n_it=100)

            ###### values ######
            y_torch_ce = torch.from_numpy(y[:, :, -1].flatten()).long()
            y_torch_acc = torch.from_numpy(y[:, :, -1])
            model_probs = torch.stack(
            [grid.X0[:, :, -1].flatten(), grid.X2[:, :, -1].flatten(), grid.X2[:, :, -1].flatten()], 
            dim=-1
            ).float()
            model_class = grid.X[:, :, -1]

            ###### metrics ######
            acc = accuracy(model_class, y_torch_acc).item()
            ce = cross_entropy(model_probs, y_torch_ce).item()
            fired = (y[:, :, -1] != 0).sum()
            model_fired = (grid.X[:, :, -1] != 0).sum().item()
            expected_fired = (1 - grid.X0[:, :, -1]).sum().item()

            metrics = [acc, ce, fired, model_fired, expected_fired]
            print(metrics)
            ######################

            df.loc[key] = metrics
        
        df.loc['Total'] = [
            df.Accuracy[:-1].mean(),
            df.CrossEntropy[:-1].mean(),
            df.Fired[:-1].mean(),
            df.ModelFired[:-1].mean(),
            df.ExpectedFired[:-1].mean()
        ]
        df.to_csv(f'data_P{i}_C{j}.csv')   

[0, 0.1, 0.5, 0.9, 1.01] 1 Artana
[0.9090220928192139, 0.6420055627822876, 2465, 4209, 3645.599853515625]
[0, 0.1, 0.5, 0.9, 1.01] 1 Beneixama
[0.9827401041984558, 0.5730377435684204, 1476, 1297, 1444.0098876953125]
[0, 0.1, 0.5, 0.9, 1.01] 1 Beniarda
[0.5249738693237305, 1.0042208433151245, 31388, 2031, 2036.9500732421875]
[0, 0.1, 0.5, 0.9, 1.01] 1 Bolulla
[0.84210205078125, 0.7095999717712402, 10400, 53, 57.679996490478516]
[0, 0.1, 0.5, 0.9, 1.01] 1 Carcaixent
[0.9367590546607971, 0.6269704699516296, 1032, 3151, 3152.18017578125]
[0, 0.1, 0.5, 0.9, 1.01] 1 Chella
[0.8984239101409912, 0.681690514087677, 315, 6543, 6637.330078125]
[0, 0.1, 0.5, 0.9, 1.01] 1 Gatova
[0.926312267780304, 0.6385226249694824, 1447, 3424, 3541.639892578125]
[0, 0.1, 0.5, 0.9, 1.01] 1 Llutxent
[0.9323835372924805, 0.6236056089401245, 2290, 2518, 2512.010009765625]
[0, 0.1, 0.5, 0.9, 1.01] 1 Montan
[0.7579978704452515, 0.8391746282577515, 5225, 11586, 11566.0]
[0, 0.1, 0.5, 0.9, 1.01] 1 ValldEbo
[0.7899438142